# Parallel Processing

In the first introductory notebook we covered the basic TOAST data and processing models, including using them interactively in a serial notebook.  In this notebook we will explore distributed data and processing.


**FIXME: reference parallel intro and document mpi4py use in jupyter**

In [ ]:
# Built-in modules
import sys

# External modules
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u

# TOAST
import toast


# Capture C++ output in the jupyter cells
%load_ext wurlitzer

# Display inline plots
%matplotlib inline

# Parallel Processing

TBD:  Major cleanup of the sections below.

### Comm : Groups of Processes

A toast.Comm instance takes the global number of processes available (MPI.COMM_WORLD) and divides them into groups.  Each process group is assigned one or more observations.  Since observations are independent, this means that different groups can be independently working on separate observations in parallel.  It also means that inter-process communication needed when working on a single observation can occur with a smaller set of processes.

At NERSC, this notebook is running on a login node, so we cannot use MPI.  Constructing a default `toast.Comm` whenever MPI use is disabled will just produce a single group of one process.  See the parallel example at the end of this notebook for a case with multiple groups.

In [ ]:
comm = toast.Comm()
print(comm)

### Data : a Collection of Observations

A toast.Data instance is mainly just a list of observations.  However remember that each process group will have a different set of observations.  Since we have only one group of one process, this example is not so interesting.  See the MPI example.

In [ ]:
data = toast.Data(comm)
data.obs.append(obs)

### Data Distribution

Recapping previous sections, we have some groups of processes, each of which has a set of observations.  Within a single process group, the detector data is distributed across the processes within the group.  That distribution is controlled by the size of the communicator passed to the TOD class, and also by the `detranks` parameter of the constructor.  This detranks number sets the dimension of the process grid in the detector direction.  For example, a value of "1" means that every process has all detectors for some span of time.  A value equal to the size of the communicator results in every process having some number of detectors for the entire observation.  The detranks parameter must divide evenly into the number of processes in the communicator and determines how the processes are arranged in a grid.

As a concrete example, imagine that MPI.COMM_WORLD has 4 processes.  We split this into 2 groups of 2 procesess.  There are 3 observations of varying lengths and every group has one or 2 observations. Here is the starting point of our data distribution:
<img src="data_dist_1.png">

Next we split the processes into 2 groups
<img src="data_dist_2.png">

Then we assign our observations to the two groups
<img src="data_dist_3.png">

When we create the TOD class in each observation, we specify how the the data is distributed within each observation.  If the `detranks` parameter is "1", then the dimension of the process grid in the detector direction is one.
<img src="data_dist_4.png">

If `detranks` is set to the size of the group, then we get:
<img src="data_dist_5.png">